# Modelo de traducción de texto
El modelo de phishing parece funcionar tanto en español como en inglés, pero por razones de confiabilidad, utilizaremos un modelo traductor primero.

In [88]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForSequenceClassification, pipeline
from torch import argmax, softmax
# Modelo de traducción
tokenizer_traductor = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-es-en")
model_traductor = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-es-en")

In [89]:
def traductor(text):
    input = tokenizer_traductor(text, return_tensors="pt", padding=True)
    output = model_traductor.generate(**input)
    traduccion = tokenizer_traductor.batch_decode(output, skip_special_tokens=True)
    return traduccion

In [90]:
# Reemplazar con la conexión a base de datos
lista_textos = [
    "Hola Mundo",
    "Si no verificas tu cuenta en las próximas 24 horas, podríamos vernos obligados a imponer restricciones adicionales o suspensor tu cuenta de forma permanente por motivos de seguridad. Proteger tu información es nuestra prioridad. Si no fue tú quien realizó esta actividad, te recomendamos que cambie tu contraseña inmediatamente después de verificar tu identidad a través del enlace propuesto.",
    "No quiero, hazlo tu"
]

traducciones_lista = traductor(lista_textos)
print(traducciones_lista)

['Hello, World.', 'If you do not check your account within the next 24 hours, we may be forced to impose additional restrictions or suspend your account permanently for security reasons. Protecting your information is our priority. If it was not you who did this activity, we recommend that you change your password immediately after verifying your identity through the proposed link.', "I don't want to, you do."]


# Modelo de detección de phishing ealvatadob

In [91]:
# Modelo de phishing
tokenizer_phishing = AutoTokenizer.from_pretrained("ealvaradob/bert-finetuned-phishing")
model_phishing = AutoModelForSequenceClassification.from_pretrained("ealvaradob/bert-finetuned-phishing")

In [92]:
def phishing(text):
    input = tokenizer_phishing(text, return_tensors="pt", padding=True)
    output = model_phishing(**input)
    logits = output.logits
    probabilidades = logits.softmax(dim=1)
    prediccion_id = argmax(probabilidades, dim=1)
    lista_resultados = []
    
    for i in range(len(prediccion_id)):
        pred_id = prediccion_id[i].item()
        etiqueta_predicha = model_phishing.config.id2label[pred_id]
        prob_predicha = probabilidades[i, pred_id].item()

        resultado = {   
            "prediccion" : "phishing" if etiqueta_predicha == "phishing" else "benigno",
            "probabilidad" : prob_predicha
        }
        lista_resultados.append(resultado)
    return lista_resultados

# Modelo de detección de Phishing ElSlay

In [ ]:
pipe = pipeline("text-classification", model="ElSlay/BERT-Phishing-Email-Model")

In [108]:
def phishing_pipe(text):
    resultado = pipe(text)
    lista_resultados = []
    for i in range(len(resultado)):
        pred_id = resultado[i]['label']
        prob_predicha = resultado[i]['score']

        resultado = {   
            "prediccion" : "phishing" if pred_id == "LABEL_1" else "benigno",
            "probabilidad" : prob_predicha
        }
        lista_resultados.append(resultado)
    return lista_resultados

# Ejemplos

In [93]:
def prediccion(text):
    traduccion = traductor(text)
    resultado = phishing(traduccion)
    return resultado

In [94]:
def correo(text):
    resultado = phishing(text)
    return resultado

In [95]:
resultado = prediccion(lista_textos)
print(resultado)

[{'prediccion': 'benigno', 'probabilidad': 0.9975121021270752}, {'prediccion': 'phishing', 'probabilidad': 0.9996534585952759}, {'prediccion': 'benigno', 'probabilidad': 0.9999877214431763}]


In [96]:
texto = "hello, delva: here's a reminder that your trial of google one (google ai pro (2 tb)) on google play will end on may 29, 2025. you will be automatically charged as described below in the payment method provided unless you cancel earlier. you can cancel your subscription and view upcoming charges at any time in the subscriptions section of google play. see how to cancel your subscription. summary next payment $79,000.00/month payment date may 29, 2025 keep this message in case you need it in the future. see google play's refund policy and terms of service. see google play order history. see google play's refund policy and terms of service you received this email reminder because you signed up for a subscription that starts with a trial. (c) 2025 google llc 1600 amphitheatre parkway, mountain view, ca 94043 do not reply to this email, as we cannot use this address to reply to you. if you need help, please visit the google play help center."

In [ ]:
# Modelo aelvatado
print(phishing(texto))

[{'prediccion': 'phishing', 'probabilidad': 0.9998579025268555}]


In [ ]:
# Modelo ElSlay
print(phishing_pipe(texto))

[{'prediccion': 'benigno', 'probabilidad': 0.9046099185943604}]


# Limpieza HTML

In [117]:
from bs4 import BeautifulSoup, Comment
import re

def clean_html_content(html_content):
    """
    Limpia el contenido HTML para extraer el texto visible. # Docstring original del usuario restaurado
    """
    if not html_content:
        return ""
    
    soup = BeautifulSoup(html_content, "html.parser")

    # Eliminar etiquetas de script y style
    for script_or_style in soup(["script", "style", "head", "meta", "link"]):
        script_or_style.decompose()

    # Eliminar comentarios HTML
    for comment_tag in soup.find_all(string=lambda text: isinstance(text, Comment)):
        comment_tag.extract() 

    # Reemplazar <br> con saltos de línea
    for br_tag in soup.find_all("br"):
        br_tag.replace_with("\n")

    # Obtener el texto
    text = soup.get_text(separator="\n", strip=True)

    # Normalizar múltiples saltos de línea y eliminar líneas vacías
    lines = [line for line in text.splitlines() if line.strip()] # Procesa las líneas para eliminar las vacías
    cleaned_text_html = "\n".join(lines) # Une las líneas limpias con un solo salto de línea
    
    # Eliminar URLs usando expresiones regulares
    # Esta expresión regular busca patrones comunes de URL (http, https, ftp, www)
    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+|www\.(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    cleaned_text_no_urls = re.sub(url_pattern, '', cleaned_text_html) # Elimina las URLs encontradas
    
    # Re-procesar líneas después de la eliminación de URLs para asegurar que todas las líneas vacías (o que quedaron vacías después del strip) se eliminen,
    # y luego colapsar múltiples saltos de línea que pudieran persistir.
    lines_after_url_removal = [line.strip() for line in cleaned_text_no_urls.splitlines()] # Divide en líneas y quita espacios en blanco de cada una
    non_empty_lines_after_url_removal = [line for line in lines_after_url_removal if line] # Filtra las líneas que quedaron completamente vacías (después de strip)
    text_reassembled = "\n".join(non_empty_lines_after_url_removal) # Vuelve a unir con un solo salto de línea
    
    # Colapso final de múltiples saltos de línea (aunque el paso anterior debería manejar la mayoría de los casos)
    # y eliminar cualquier espacio en blanco al inicio o final de todo el texto.
    cleaned_text_final = re.sub(r'\n{2,}', '\n', text_reassembled).strip() 
    
    return cleaned_text_final # Devuelve el texto procesado, sin URLs y con saltos de línea normalizados

ModuleNotFoundError: No module named 'bs4'